<h1>Rotulação de Dados</h1>

Neste notebook é criado labels para os dados.

<h4>Rotulação de Tipo de Superfície</h4>

<table>
     <tr>
         <th>Valor</th>
         <th>Tipo</th>
         <th>Especificação</th>
     </tr>
     <tr>
         <td>0,1</td>
         <td>Sem Pavimento</td>
         <td>Terra</td>
     </tr>
     <tr>
         <td>0,1</td>
         <td>Pavimento Flexível</td>
         <td>Paralelepípedo/Hexagonal</td>
     </tr>
     <tr>
         <td>0,1</td>
         <td>Pavimento Rígido</td>
         <td>Asfalto</td>
     </tr>
</table>


<h4>Importação de Pacotes</h4>

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil import parser
# %matplotlib inline
%matplotlib notebook
pd.set_option("float_format", '{:0.10f}'.format)
pd.set_option('display.max_columns', 30)
%run "3 - Data Labeling.py"

<h4>Funções Utilitárias</h4>

In [ ]:
# Transforma Date String em Timestamp Int
def parseDateToTimestamp(value):
    newDate = parser.parse(value)
    return datetime.timestamp(newDate)

# Transforma Timestamp Int em Date String 
def parseTimestampToDate(value):
    newDate = datetime.fromtimestamp(value)
    return newDate.strftime("%Y-%m-%dT%H:%M:%S")

# Transforma Array Tupla de Date para Array Tupla de Timestamp
def parseDateToTimestampTuple(values):
    
    result = []

    for i in values:
        x = parseDateToTimestamp(i[0])
        y = parseDateToTimestamp(i[1])
        result.append((x,y))

    return result

# Rotula cada amostra com base nos labels e timestamp
def mapToLabel(timestamp, ranges_timestamp):
    
    for interval in ranges_timestamp:
        
        if(timestamp >= interval[0] and timestamp <= interval[1]):
            return 1
        
    return 0

# Adiciona uma classe de dado como coluna no dataset
def addClassToData(classes, data_mpu, ranges_date, class_name):
    timestamps = data_mpu['timestamp'].copy()
    ranges_timestamp = parseDateToTimestampTuple(ranges_date)
    class_values = timestamps.apply(mapToLabel, args=(ranges_timestamp,))
    classes.insert(len(classes.columns), class_name, class_values, True)
    
# Método principal 
def processLabels(data_mpu, ranges_date):
    
    classes = pd.DataFrame()
    
    addClassToData(classes, data_mpu, ranges_date['sem_pavimento'], 'land')
    addClassToData(classes, data_mpu, ranges_date['pavimento_flexivel'], 'cobblestone')
    addClassToData(classes, data_mpu, ranges_date['pavimento_rigido'], 'asphalt')
    
    file = os.path.join(folder, 'dataset_labels.csv')
    classes.to_csv(file, index=False)
    print("Salvo em ", file)

    return classes

# Plot dados com classes
def plotClasses(data_mpu, classes):
    plt.figure(figsize=(16,6)) 
    data_mpu['acc_z_below_suspension'].rolling(5).std().plot(color="y")
    # data_mpu['gyro_y_below_suspension'].plot()
    # data_mpu['gyro_x_below_suspension'].plot()
    (data_mpu['speed'] * 3.6).plot()
    (classes['land'] * 20).plot(linewidth=2)
    (classes['cobblestone'] * 40).plot(linewidth=2)
    (classes['asphalt'] * 60).plot(linewidth=2)
    plt.legend()

<h4>Importação dos Dados</h4>

In [ ]:
settings = workspace['settings']
dataset = workspace['datasets']['saveiro-3']

folder = os.path.join(settings['folder'], dataset['folder'])
data_esquerda = pd.read_csv(os.path.join(folder, 'dataset_gps_mpu_left.csv'), float_precision="high")
data_direita = pd.read_csv(os.path.join(folder, 'dataset_gps_mpu_right.csv'), float_precision="high")

<h4>Rotulação de Dados</h4>

In [ ]:
classes = processLabels(data_esquerda.copy(), dataset.copy())

In [ ]:
plotClasses(data_esquerda, classes)

In [ ]:
plotClasses(data_direita, classes)